# IBM Applied Data Science Capstone Project by Coursera

**Opening a new coffee shop in Bengaluru, India**

* Build a data frame of neighborhoods in Bengaluru by scraping the web data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new coffee shop

### 1. Import Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geopy
!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
!pip install bs4
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a data frame

In [5]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [6]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [7]:
# create a list to store neighborhood data
neighborhoodList = []

In [8]:
# append the data into the list
for row in soup.find_all("div", class_ = "mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [9]:
# create a new DataFrame from the list
blore_df = pd.DataFrame({"Neighborhood" : neighborhoodList})

blore_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [10]:
blore_df.drop(blore_df.index[[0,1,2]], inplace = True) # drop the first 3 rows as they do not correspond to neighborhood names

In [11]:
df = blore_df.reset_index (drop = True) # reset the index to start the third neighborhood name from first index
df.head()

,Neighborhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere


In [12]:
# print the number of rows of the dataframe
df.shape

(128, 1)

### 3. Obtain the geographical coordinates

In [13]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bengaluru, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist()]

In [15]:
coords

[[12.944020000000023, 77.60800000000006],
 [12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.02753000000007, 77.65049000000005],
 [13.07728933897749, 77.65760586439058],
 [13.044710000000066, 77.55008000000004],
 [12.922310000000039, 77.56988000000007],
 [13.019646729517714, 77.65469401219607],
 [12.938980000000072, 77.57137000000006],
 [12.992220000000032, 77.53444000000007],
 [12.927350000000047, 77.67185000000006],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.91488000000004, 77.61004000000008],
 [12.951940000000036, 77.54445000000004],
 [12.913110000000074, 77.71565000000004],
 [12.959697329772126, 77.57131161084145],
 [12.794010000000071, 77.700150000000

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns = ['Latitude', 'Longitude'])

In [17]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [18]:
# check the neighborhoods and the coordinates
print(df.shape)
df

(128, 3)


,Neighborhood,Latitude,Longitude
0,Adugodi,12.944020,77.608000
1,"Agara, Bangalore",12.842830,77.487590
2,Ananthnagar,12.954080,77.541350
3,Anjanapura,12.858110,77.559090
4,Arekere,12.885680,77.596680
5,Austin Town,12.963480,77.612970
6,Babusapalya,13.027530,77.650490
7,"Bagalur, Bangalore Urban",13.077289,77.657606
8,Bahubalinagar,13.044710,77.550080
9,Banashankari,12.922310,77.569880


In [19]:
# save the DataFrame as CSV file
df.to_csv("blore_area.csv", index = False)

### 4. Create a map of Bengaluru with the neighborhoods superimposed on it

In [20]:
# get the coordinates of Bengaluru
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent = "blore_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru, India are 12.9791198, 77.5912997.


In [21]:
# create map of Bengaluru using latitude and longitude values
map_blore = folium.Map(location = [latitude, longitude], zoom_start = 12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_blore)  
    
map_blore

In [22]:
# save the map as HTML file
map_blore.save('map_blore.html')

### 5. Use Foursquare API to explore the neighborhoods

In [23]:
# define Foursquare Credentials and Version
CLIENT_ID = 'DPWRMW0IYL0LY3SY3RTXG03JT0FPMOWA12DQO12KVVULFIQV' #Foursquare Client ID
CLIENT_SECRET = 'E20KMIHOUYXJDMUISXCA34OJ5QNHSSIDQTFKSVL1AQPARRGI' #Foursquare Client Secret
VERSION = '20200629' #Foursquare API Version

print('Your credentails:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID:DPWRMW0IYL0LY3SY3RTXG03JT0FPMOWA12DQO12KVVULFIQV
CLIENT_SECRET:E20KMIHOUYXJDMUISXCA34OJ5QNHSSIDQTFKSVL1AQPARRGI


In [24]:
# borrow the get_category_type function from the Foursquare lab
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

*Retrieve the top 100 venues that are within a radius of 2 kms*

In [25]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6709, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94402,77.608,Tommy Hilfiger,12.934552,77.611347,Clothing Store
1,Adugodi,12.94402,77.608,PVR IMAX,12.934595,77.611321,Movie Theater
2,Adugodi,12.94402,77.608,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
3,Adugodi,12.94402,77.608,Lot Like Crêpes,12.936421,77.613284,Creperie
4,Adugodi,12.94402,77.608,Fenny's Lounge & Kitchen,12.935146,77.613536,Lounge


In [27]:
venues_df.groupby(["Neighborhood"]).count() # number of venues returned for each neighborhood

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,24,24,24,24,24,24
Anjanapura,5,5,5,5,5,5
Arekere,83,83,83,83,83,83
Austin Town,100,100,100,100,100,100
BTM Layout,87,87,87,87,87,87
Babusapalya,49,49,49,49,49,49
"Bagalur, Bangalore Urban",8,8,8,8,8,8
Bahubalinagar,19,19,19,19,19,19


In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique()))) # unique categories from the above venues

There are 234 uniques categories.


In [29]:
venues_df['VenueCategory'].unique()

array(['Clothing Store', 'Movie Theater', 'Burger Joint', 'Creperie',
       'Lounge', 'Indian Restaurant', 'Brewery', 'Dessert Shop',
       'Multiplex', 'Financial or Legal Service', 'Tea Room', 'Bar',
       'Café', 'Breakfast Spot', 'Mobile Phone Shop', 'Shopping Mall',
       'Gym', 'Donut Shop', 'Coffee Shop', 'Juice Bar', 'Ice Cream Shop',
       'Italian Restaurant', 'Chinese Restaurant', 'Bakery',
       'Indian Sweet Shop', "Men's Store", 'Yoga Studio', 'Smoke Shop',
       'Arcade', 'Gaming Cafe', 'Andhra Restaurant', 'Bookstore',
       'Kerala Restaurant', 'Fast Food Restaurant', 'Pizza Place',
       'Mexican Restaurant', 'Cosmetics Shop', 'Bowling Alley',
       'Tibetan Restaurant', 'Comfort Food Restaurant', 'Bistro',
       'Punjabi Restaurant', 'Eastern European Restaurant',
       'Thai Restaurant', 'Persian Restaurant',
       'Paper / Office Supplies Store', 'Seafood Restaurant',
       'Restaurant', 'Sandwich Place', 'Liquor Store', 'Snack Place',
       'Middle 

### 6. Analyze each neighborhood

In [30]:
# one hot encoding
blore_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
blore_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blore_onehot.columns[-1]] + list(blore_onehot.columns[:-1])
blore_onehot = blore_onehot[fixed_columns]

print(blore_onehot.shape)
blore_onehot.head()

(6709, 235)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
blore_grouped = blore_onehot.groupby(["Neighborhoods"]).mean().reset_index() # mean of frequency of each category in all neighborhoods

print(blore_grouped.shape)
blore_grouped

(127, 235)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.030000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.

In [32]:
len(blore_grouped[blore_grouped["Coffee Shop"] > 0]) # length of the Coffee Shop column

94

In [33]:
blore_rest = blore_grouped[["Neighborhoods", "Coffee Shop"]] # create a new data frame for Coffee Shop only

In [34]:
blore_rest.head()

,Neighborhoods,Coffee Shop
0,Adugodi,0.030000
1,Ananthnagar,0.083333
2,Anjanapura,0.200000
3,Arekere,0.012048
4,Austin Town,0.010000


### 7. Cluster neighborhoods

*Runs k - means to cluster the neighborhoods in Bengaluru into 5 clusters*

In [35]:
# set number of clusters
kclusters = 3

blore_clustering = blore_rest.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(blore_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 2, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

In [36]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
blore_merged = blore_rest.copy()

# add clustering labels
blore_merged["Cluster Labels"] = kmeans.labels_

In [37]:
blore_merged.rename(columns = {"Neighborhoods" : "Neighborhood"}, inplace = True)
blore_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Adugodi,0.030000,1
1,Ananthnagar,0.083333,0
2,Anjanapura,0.200000,2
3,Arekere,0.012048,1
4,Austin Town,0.010000,1


In [38]:
# merge blore_grouped with df to add latitude/longitude for each neighborhood
blore_merged = blore_merged.join(df.set_index("Neighborhood"), on = "Neighborhood")

print(blore_merged.shape)
blore_merged.head()

(127, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Adugodi,0.030000,1,12.94402,77.60800
1,Ananthnagar,0.083333,0,12.95408,77.54135
2,Anjanapura,0.200000,2,12.85811,77.55909
3,Arekere,0.012048,1,12.88568,77.59668
4,Austin Town,0.010000,1,12.96348,77.61297


In [39]:
# sort the results by Cluster Labels
blore_merged.sort_values(["Cluster Labels"], inplace = True)

In [40]:
blore_merged.reset_index(drop = True)

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Yeswanthpur,0.076923,0,13.029540,77.540220
1,Shikaripalya,0.078947,0,12.832600,77.656850
2,Domlur,0.066667,0,12.943290,77.656020
3,Seshadripuram,0.060000,0,12.993550,77.579880
4,Electronic City,0.096154,0,12.845470,77.664300
5,Kundalahalli,0.050000,0,12.967520,77.715000
6,Ramachandrapura,0.062500,0,13.061240,77.551120
7,"Rajarajeshwari Nagar, Bangalore",0.060606,0,12.931780,77.526680
8,Hebbal,0.051282,0,13.049810,77.589030
9,Hoodi,0.081633,0,12.991980,77.715060


*Visualize the resulting clusters*

In [41]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blore_merged['Latitude'], blore_merged['Longitude'], blore_merged['Neighborhood'], blore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [42]:
# save the map as HTML file
map_clusters.save('blore_map_clusters.html')

### 8. Examine the clusters

*CLUSTER 0*

In [43]:
blore_merged.loc[blore_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
126,Yeswanthpur,0.076923,0,13.029540,77.540220
103,Shikaripalya,0.078947,0,12.832600,77.656850
31,Domlur,0.066667,0,12.943290,77.656020
102,Seshadripuram,0.060000,0,12.993550,77.579880
33,Electronic City,0.096154,0,12.845470,77.664300
72,Kundalahalli,0.050000,0,12.967520,77.715000
94,Ramachandrapura,0.062500,0,13.061240,77.551120
93,"Rajarajeshwari Nagar, Bangalore",0.060606,0,12.931780,77.526680
44,Hebbal,0.051282,0,13.049810,77.589030
45,Hoodi,0.081633,0,12.991980,77.715060


*CLUSTER 1*

In [44]:
blore_merged.loc[blore_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
85,Nandini Layout,0.034884,1,13.014810,77.538910
84,Nagarbhavi,0.000000,1,12.956240,77.509360
83,Muthyalanagar,0.000000,1,13.041030,77.551480
82,"Murphy Town, Bangalore",0.022989,1,12.980030,77.627760
81,Milk Colony,0.032609,1,13.023190,77.579060
108,Suddaguntepalya,0.034884,1,12.983890,77.654760
78,Marathahalli,0.027778,1,12.954660,77.707520
87,Padmanabhanagar,0.000000,1,12.918140,77.559400
122,"Whitefield, Bangalore",0.028986,1,12.979380,77.733720
76,Mahalakshmi Layout,0.046875,1,13.016350,77.544810


*CLUSTER 2*

In [45]:
blore_merged.loc[blore_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
23,Chandapura,0.25,2,12.79401,77.70015
46,Horamavu,0.25,2,13.03870,77.66192
2,Anjanapura,0.20,2,12.85811,77.55909


**Observations:**

Most of the coffee shops are concentrated in the southern, eastern and central parts of the Bengaluru city, with the highest numbers in cluster 2 and moderate numbers in cluster 0. On the other hand, cluster 1 has very low number to no coffee shops in the neighborhoods. This cluster represents a great opportunity and high potential areas to open new coffee shops as there is very little to no competition from ones already present. Meanwhile, coffee shops in cluster 2 are likely suffering from intense competition due to high concentration of other coffee shops. Therefore, this project recommends coffee business owners and popular coffee chains to capitalize on these findings to open new coffee shops in the neighborhoods in cluster 1. Coffee business owners with unique selling point (USP), stand out from the competition and can also open new coffee shops in the neighborhoods in clusters 0 with moderate competition.

# -------------------------------End of the notebook-------------------------------